In [1]:
import numpy as np
import scipy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import unidecode

In [2]:
%load_ext autoreload
%autoreload 2

In [66]:
ingred_df = pd.read_csv('../data/interim/Agribalyse_Detail ingredient.csv')
etapes_df = pd.read_csv('../data/interim/Agribalyse_Detail etape.csv')

In [67]:
ingred_df.sample(3)

,Unnamed: 0,Code_AGB,Code_CIQUAL,Groupe_aliment,Sous-groupe_aliment,Nom_Produit_Francais,LCI_Name,Saisonnalite,Transport_par_avion_,Livraison,...,Viande de moutton sans os,Viande de porc maigre,Viande de poulet sans os,Viande de veau sans os,Vin blanc,Vin rouge,Yaourt,citron,Échalote,Œuf de poule
492,492,16614,16614,matières grasses,margarines,Matière grasse végétale (type margarine) à 80%...,"Vegetable fat (like margarine), 80% fat, salted",mix de consommation FR,0,Glacé,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121,121,13161,13161,aliments infantiles,laits et boissons infantiles,Boisson infantile céréales lactées aux légumes...,Dairy cereal-based beverage with vegetables fo...,mix de consommation FR,0,Ambiant (long),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
301,301,7650,7650,produits céréaliers,pains et viennoiseries,"Croissant aux amandes, artisanal","Croissant w almonds, from bakery",mix de consommation FR,0,Ambiant (court),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.042432


In [59]:
def missing(df):
    missing = ingred_df.isna().sum()
    missing[missing>0]

In [71]:
missing(ingred_df)
ingred_df.fillna(0, inplace=True)


In [72]:
missing(ingred_df)
ingred_df

,Unnamed: 0,Code_AGB,Code_CIQUAL,Groupe_aliment,Sous-groupe_aliment,Nom_Produit_Francais,LCI_Name,Saisonnalite,Transport_par_avion_,Livraison,...,Viande de moutton sans os,Viande de porc maigre,Viande de poulet sans os,Viande de veau sans os,Vin blanc,Vin rouge,Yaourt,citron,Échalote,Œuf de poule
0,0,25433,25433,"viandes, œufs, poissons",produits à base de poissons et produits de la mer,Accra de poisson,"Caribbean-style fish fritters, fish acras",mix de consommation FR,0,Glacé,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.046785
1,1,4041,4041,entrées et plats composés,plats composés,Aligot (purée de pomme de terre à la tomme fra...,Mashed potatoes w fresh tome cheese,mix de consommation FR,0,Glacé,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,2,26187,26187,"viandes, œufs, poissons",produits à base de poissons et produits de la mer,"Anchois commun, mariné","Common anchovy, marinated",mix de consommation FR,0,Glacé,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,3,26000,26000,"viandes, œufs, poissons",produits à base de poissons et produits de la mer,"Anchois, filets à l'huile, semi-conserve, égoutté","Anchovy, fillets, in oil, semi-preserved, drained",mix de consommation FR,0,Glacé,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,4,25999,25999,"viandes, œufs, poissons",produits à base de poissons et produits de la mer,"Anchois, filets roulés aux câpres, semi-conser...","Anchovy, fillets, rolled with capers, semi-pre...",mix de consommation FR,0,Glacé,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1033,1033,19592,19592,lait et produits laitiers,produits laitiers frais et assimilés,"Yaourt, lait fermenté ou spécialité laitière, ...","Yogurt, fermented milk or dairy specialty, wit...",mix de consommation FR,0,Glacé,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1034,1034,19593,19593,lait et produits laitiers,produits laitiers frais et assimilés,"Yaourt, lait fermenté ou spécialité laitière, ...","Yogurt, fermented milk or dairy specialty, plain",mix de consommation FR,0,Glacé,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1035,1035,19594,19594,lait et produits laitiers,produits laitiers frais et assimilés,"Yaourt, lait fermenté ou spécialité laitière, ...","Yogurt, fermented milk or dairy specialty, pla...",mix de consommation FR,0,Glacé,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1036,1036,19598,19598,lait et produits laitiers,produits laitiers frais et assimilés,"Yaourt, lait fermenté ou spécialité laitière, ...","Yogurt, fermented milk or dairy specialty, pla...",mix de consommation FR,0,Glacé,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [73]:
only_ingred = ingred_df.iloc[:, 15:]
only_ingred

,Abats de bœuf,Abats de porc,Abats de poulet,Abats de veau,Abricot,Ail,Amande,Amidon de maïs,Ananas,Anchois,...,Viande de moutton sans os,Viande de porc maigre,Viande de poulet sans os,Viande de veau sans os,Vin blanc,Vin rouge,Yaourt,citron,Échalote,Œuf de poule
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.046785
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.114290,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.109798,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.109798,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1033,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1035,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1036,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [77]:
only_ingred[only_ingred['Abats de porc']>0]
for i,c in enumerate(only_ingred.columns.to_list()):
    if i == 1:
        only_ingred.loc[only_ingred[only_ingred[c] > 0].index, c] = 1
        print(c)
    break

In [78]:
only_ingred[only_ingred['Abats de bœuf'] > 0]

,Abats de bœuf,Abats de porc,Abats de poulet,Abats de veau,Abricot,Ail,Amande,Amidon de maïs,Ananas,Anchois,...,Viande de moutton sans os,Viande de porc maigre,Viande de poulet sans os,Viande de veau sans os,Vin blanc,Vin rouge,Yaourt,citron,Échalote,Œuf de poule
5,0.463192,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
6,0.463192,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
7,0.463192,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
8,0.463192,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
9,0.463192,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
10,0.463192,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
11,0.463192,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
139,0.463192,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
140,0.707618,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
141,0.707618,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
